In [2]:
# -*- coding: UTF-8 -*-
import sys
import time
import leveldb
from urllib import quote_plus
import re
import json
import itertools
import sys
import requests
from Queue import Queue
from threading import Thread

URL_BASE = 'http://s.m.taobao.com/search?q={}&n=200&m=api4h5&style=list&page={}'

def url_get(url):
    # print('GET ' + url)
    header = dict()
    header['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
    header['Accept-Encoding'] = 'gzip,deflate,sdch'
    header['Accept-Language'] = 'en-US,en;q=0.8'
    header['Connection'] = 'keep-alive'
    header['DNT'] = '1'
    # header['User-Agent'] = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.71 Safari/537.36'
    header['User-Agent'] = 'Mozilla/12.0 (compatible; MSIE 8.0; Windows NT)'
    return requests.get(url, timeout=5, headers=header).text

In [4]:
reload(sys)
sys.setdefaultencoding('utf-8')

wx_id = 'wx000001'
item_name = '壹可医研弹力美肌水'
item_page = '1'


In [19]:
import redis
pool = redis.ConnectionPool(host='localhost',port=6379,decode_responses=True)
rds = redis.Redis(connection_pool=pool)
rds.set('leveldb_id', '1')
rds.get('leveldb_id')

In [14]:
url = URL_BASE.format(quote_plus(item_name), item_page)
print(url)

In [15]:
for tr in range(5):
    try:
        items_obj = json.loads(url_get(url))
        break
    except KeyboardInterrupt:
        quit()
    except Exception as e:
        if tr == 4: raise e
        
if len(items_obj['listItem']) == 0:
    print('no listItem')


In [22]:
for item in items_obj['listItem']:
    item_obj = dict(
        _id=int(item['itemNumId']),
        name=item['name'],
        wx_id=wx_id,
        nick=item['nick']
    )
    id = rds.incr('leveldb_id')
    rds.set(str(id), json.dumps(item_obj, ensure_ascii=False))
print('Get {} items from {} on page {}'.format(len(items_obj['listItem']), item_name, item_page))




In [23]:
for k in rds.keys():
    print k
    print rds.get(k)

In [86]:
item_id = 542641956354

In [88]:
base_url='https://h5api.m.taobao.com/h5/mtop.taobao.baichuan.smb.get/1.0/?jsv=2.4.16&appKey=12574478&t=1540469490629&sign=52f5345f9f23e070d506a864e5ba09cd&api=mtop.taobao.baichuan.smb.get&v=1.0&type=originaljson&dataType=jsonp&timeout=10000'

In [100]:
header = dict()
header['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
header['Accept-Encoding'] = 'gzip,deflate,sdch'
header['Accept-Language'] = 'en-US,en;q=0.8'
header['Connection'] = 'keep-alive'
header['DNT'] = '1'
header['User-Agent'] = 'Mozilla/5.0 (iPhone; CPU iPhone OS 11_0 like Mac OS X) AppleWebKit/604.1.38 (KHTML, like Gecko) Version/11.0 Mobile/15A372 Safari/604.1'
header['referer'] = 'https://market.m.taobao.com/app/mtb/h5-tb-detail-old/index.html?ut_sk=1.WIcwks7gRNMDAL1z8%2BpHckJi_21380790_1540282964443.AliPay.1&id=560730435940&sourceType=item&price=917&suid=D04AC4D7-9F45-42F0-8274-53A5C785F008'
header['cookie'] = 'https://market.m.taobao.com/app/mtb/h5-tb-detail-old/index.html?ut_sk=1.WIcwks7gRNMDAL1z8%2BpHckJi_21380790_1540282964443.AliPay.1&id=560730435940&sourceType=item&price=917&suid=D04AC4D7-9F45-42F0-8274-53A5C785F008' 
print requests.get(base_url, timeout=5, headers=header).text




In [46]:
try:
    db_cate.Get(orig_cate)
except:
    db_cate.Put(orig_cate, 'waiting')

In [ ]:
cate_queue = Queue(maxsize=1000)

In [50]:
cate_th = Thread(target=cate_thread, args=(cate_queue, db_cate))
cate_th.start()


In [2]:
url = URL_BASE.format(quote_plus(cate), item_page)

NameError: name 'URL_BASE' is not defined

In [ ]:
def item_thread(item_name, db_item):
    while True:
        try:
            cate = cate_queue.get()
            post_exist = True
            try:
                state = db_cate.Get(cate)
                if state != 'OK': post_exist = False
            except:
                post_exist = False
            if post_exist == True:
                print('cate-{}: {} already exists ... Ignore'.format(cate, cate))
                continue
            db_cate.Put(cate, 'crawling')
            for item_page in itertools.count(1):
                url = URL_BASE.format(quote_plus(cate), item_page)
                print(url)
                for tr in range(5):
                    try:
                        items_obj = json.loads(url_get(url))
                        break
                    except KeyboardInterrupt:
                        quit()
                    except Exception as e:
                        if tr == 4: raise e
                #----jump out from loop-----#
                # break when page is empty or
                if len(items_obj['listItem']) == 0 or item_page > 1: break
                for item in items_obj['listItem']:
                    item_obj = dict(
                        _id=int(item['itemNumId']),
                        name=item['name'],
                        price=float(item['price']),
                        query=cate,
                        # category = int(item['category']) if item['category'] != '' else 0,
                        nick=item['nick'],
                        area=item['area'])
                    db_item.Put(str(item_obj['_id']).encode('utf-8'),
                                json.dumps(item_obj, ensure_ascii=False))

                print('Get {} items from {} on page {}'.format(len(items_obj['listItem']), cate, item_page))

                if 'nav' in items_obj:
                    for na in items_obj['nav']['navCatList']:
                        try:
                            db_cate.Get(na['name'])
                        except:
                            db_cate.Put(na['name'], 'waiting')
            db_cate.Put(cate, 'OK')
            print(cate.encode('utf-8'), 'OK')
            break
        except KeyboardInterrupt:
            break
        except Exception as e:
            print('An {} exception occured'.format(e))


True

In [28]:
rds.set('1111', 'a')
print rds.get('1111')
rds.set('1111', 'aaa')
print rds.get('1111')